# Statistiques descriptives

Les données sont le résultat d'un contôle. Il y a 26 étudiants et pour chacun sont listés le temps mit pour faire l'épreuve et le nombre d'erreurs.

**On regarde** le fichier dans un éditeur de texte :

* encodage : unicode
* délimiteur : espace et/ou tabulation

In [ ]:
import pandas

In [ ]:
epreuve = pandas.read_csv("./epreuve.txt", delim_whitespace=True)
epreuve

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.scatterplot.html

fig, ax = plt.subplots(figsize=(10, 10))

sns.scatterplot(x=epreuve['temps'], 
                y=epreuve['erreurs'],
                ax=ax)
for i in epreuve.index:
    ax.text(epreuve['temps'][i], epreuve['erreurs'][i], str(i))
plt.show()

Comme pour des données catégorielles, pour voir les éléments supperposés on peut ajouter un *jitter*.  Mais attention, ce ne sont plus les données originelles représentées comme des point d'un espace vectoriel.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.stripplot(x=epreuve['temps'], 
              y=epreuve['erreurs'], 
              ax=ax)
plt.show()

Pour les labels, c'est un peu plus compliqué. Ci-dessous une solution possible.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.stripplot(x=epreuve['temps'], 
              y=epreuve['erreurs'], 
              ax=ax
             )
temps_values = epreuve['temps'].drop_duplicates().sort_values().to_list()
decalage = {}
for i in epreuve.index:
    x, y = temps_values.index(epreuve['temps'][i]), epreuve['erreurs'][i]
    if (x, y) not in decalage:
        decalage[(x, y)] = 0
    else:
        decalage[(x, y)] += .1
    ax.text(x + decalage[(x, y)], y, str(i))

plt.show()

# Premières analyses

## résumé des données

### Par des séquences de nombres

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5), ncols=2) # 2 histogrammes, un sur chaque colonne
epreuve.hist(ax=ax)
plt.show()

### Par des densités

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.histplot(data=epreuve, x='temps', kde=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.histplot(data=epreuve, x='erreurs', kde=True, ax=ax)
plt.show()

### Par des nombres

In [ ]:
epreuve.describe()

On peut représentez ces nombres dans [une boîte à moustache](https://en.wikipedia.org/wiki/Box_plot) :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.boxplot(data=epreuve, x='temps', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.boxplot(data=epreuve, x='erreurs', ax=ax)
plt.show()

Ou encore une [violin box](https://en.wikipedia.org/wiki/Violin_plot) :

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.violinplot(data=epreuve, x='temps', ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.violinplot(data=epreuve, x='erreurs', ax=ax)
plt.show()

## Qu'est ce que la moyenne ?

Résumer les données par 1 unique chiffre. Lequel ?
Si $x = (x_1, \dots, x_n)$ on cherche $\bar{x}$ le plus proche de tous les points (en norme $L_2$):
$$\min_{y} \frac{1}{n} \sum_{i=1}^{n} (x_i - y)^2$$
On trouve que $y = \bar{x} = \sum_{i=1}^n x_i$. C'est la moyenne.

In [ ]:
moyenne = epreuve.mean()
print(moyenne)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.scatterplot(x=epreuve['temps'], 
                y=epreuve['erreurs'],
                ax=ax)

# for i in epreuve.index:
#     ax.text(epreuve['temps'][i], epreuve['erreurs'][i], str(i))

ax.scatter(moyenne['temps'], moyenne['erreurs'], marker="+", color="red")
plt.show()

Si on avait pris une autre norme (par exemple $L_1$) on aurait été amené à minimiser la quantité $$\min_{y} \frac{1}{n}\sum_{i=1}^{n} |x_i - y|$$
On trouve que dans ce cas là, $y$ est égal à la médiane $\mbox{m}(x)$.

In [ ]:
mediane = epreuve.median()
print(mediane)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.scatterplot(x=epreuve['temps'], 
                y=epreuve['erreurs'],
                ax=ax)

# for i in epreuve.index:
#     ax.text(epreuve['temps'][i], epreuve['erreurs'][i], str(i))

ax.scatter(mediane['temps'], mediane['erreurs'], marker="*", color="purple")
ax.scatter(moyenne['temps'], moyenne['erreurs'], marker="+", color="red")

plt.show()

# Centrer et réduire les données

## centrer

L'origine peut être très éloignée des données. On les recentre autour de la moyenne.

In [ ]:
epreuve_centree = epreuve - moyenne

epreuve_centree

In [ ]:
epreuve_centree.mean()

Les données centrées permettent de voir ce qui est petit / grand par rapport à la moyenne

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(x=epreuve_centree['temps'], 
                y=epreuve_centree['erreurs'],
                ax=ax)

for i in epreuve.index:
    ax.text(epreuve_centree['temps'][i], epreuve_centree['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)
plt.show()

## réduire

Pour savoir comment se comportent les objets par rapport à leurs coordonnées on a un problème d'unité : temps et nombre d'erreurs. Plus pertinent de regarder l'écart par rapport à la position moyenne.

L'erreur des nombres par rapport à la moyenne est appelé variance : $\frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2 = \sigma(x)^2$.
De là si on divise chaque nombre par $\sigma$ (appelé l'écart type) on obtient $x'_i = \frac{x_i}{\sigma}$ et de là la variance de $x' = (x_1, \dots, x_n)$ vaut $1$ et est un nombre sans unité : on homogénise les différents axes.  


In [ ]:
ecart_type =  epreuve_centree.std()

ecart_type

In [ ]:
epreuve_centree_reduit = epreuve_centree / ecart_type

epreuve_centree_reduit

In [ ]:
epreuve_centree_reduit.std()

Les données centrée et réduite sont dite homogénéisées. On peut les comparer car ils ont la même unité (sans unité) et leur variantion est identique (variance = écart à la moyenne = 1).
Cette opération étant quasi-obligatoire pour tout jeu de données, une méthode toute faite existe.

## sklearn

Sklearn https://scikit-learn.org/stable/ est une bibliothèque très utilisée en machine learning. Nous allons l'utiiser dansune optique d'analyse des données.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
epreuve_scaled_sklearn = scaler.fit_transform(pandas.DataFrame(epreuve, dtype='float')) # sinon warning de conversion

Sklearn nous a rendu un tableau et pas un dataframe de pandas. Remédions à ça en recréeant un dataframe

In [ ]:
epreuve_scaled = pandas.DataFrame(epreuve_scaled_sklearn, columns=epreuve.columns)
epreuve_scaled

In [ ]:
epreuve_scaled.describe()

On peut maintenant trouver les étudiants ayant fait moins bien que la moyenne en temps et en erreurs.

Au delà de ±2 c'est un gros écrart.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(x=epreuve_scaled['temps'], 
                y=epreuve_scaled['erreurs'],
                ax=ax)

for i in epreuve.index:
    ax.text(epreuve_scaled['temps'][i], epreuve_scaled['erreurs'][i], str(i))

ax.axvline(0)
ax.axhline(0)
plt.show()